In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score

In [2]:
dataframe = pd.read_csv('salida/resultadoMRITDEVCOMPLETO_9500.csv')

In [3]:
dataframe

,suma,distancias,entropias,mutual_info,m_earth,max_info_p,antonimos,relaciones,diferencias,CLASS
0,1.032591,262.348270,2.322,9.185561,0.019789,0.206518,0,0,5.000000,neutral
1,0.000000,51.972244,0.000,0.000000,0.000000,NaN,0,0,1.000000,entailment
2,0.078171,138.414410,3.000,3.586406,0.012995,0.386005,0,1,0.500000,contradiction
3,0.045913,117.188370,2.948,1.893555,0.008837,0.757188,0,0,0.111111,entailment
4,0.028080,152.295070,3.122,1.892166,0.009850,0.383457,0,0,0.100000,neutral
...,...,...,...,...,...,...,...,...,...,...
9496,0.608029,87.016080,1.500,3.539328,0.018841,0.448370,0,1,1.000000,contradiction
9497,0.790268,169.100980,1.585,5.479692,0.014555,0.263423,0,0,3.000000,neutral
9498,0.000000,67.233696,0.000,0.000000,0.000000,0.000000,0,0,0.500000,entailment
9499,0.540114,141.756090,2.585,5.266223,0.018192,0.435303,1,0,1.500000,contradiction


In [4]:
#dataframe=dataframe.drop(['mutual_info'],axis=1)

In [5]:
dataframe

,suma,distancias,entropias,mutual_info,m_earth,max_info_p,antonimos,relaciones,diferencias,CLASS
0,1.032591,262.348270,2.322,9.185561,0.019789,0.206518,0,0,5.000000,neutral
1,0.000000,51.972244,0.000,0.000000,0.000000,NaN,0,0,1.000000,entailment
2,0.078171,138.414410,3.000,3.586406,0.012995,0.386005,0,1,0.500000,contradiction
3,0.045913,117.188370,2.948,1.893555,0.008837,0.757188,0,0,0.111111,entailment
4,0.028080,152.295070,3.122,1.892166,0.009850,0.383457,0,0,0.100000,neutral
...,...,...,...,...,...,...,...,...,...,...
9496,0.608029,87.016080,1.500,3.539328,0.018841,0.448370,0,1,1.000000,contradiction
9497,0.790268,169.100980,1.585,5.479692,0.014555,0.263423,0,0,3.000000,neutral
9498,0.000000,67.233696,0.000,0.000000,0.000000,0.000000,0,0,0.500000,entailment
9499,0.540114,141.756090,2.585,5.266223,0.018192,0.435303,1,0,1.500000,contradiction


In [6]:
X = np.array(dataframe.drop(['CLASS'],axis=1))
y_ = np.array(dataframe['CLASS'])

In [7]:
y=[]
for i in y_:
    if i == "contradiction":
        y.append(0)
    elif i == "entailment":
        y.append(1)
    else:
        y.append(2)

In [8]:
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)
X = X_scaled

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=7)

In [10]:
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

In [11]:
import xgboost as xgb

In [12]:
# Define hyperparameters
#params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}

In [13]:
clf_xgb = xgb.XGBClassifier(objective='reg:logistic',n_estimators=1000,device="cuda")
clf_xgb.fit(X_train, y_train)
preds_xgb = clf_xgb.predict(X_test)

/home/david/anaconda3/envs/rit/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [18:47:33] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [14]:
accuracy_score(y_test,preds_xgb)

0.5286691215149921

In [15]:
clf_lr = LR().fit(X_train, y_train)

In [16]:
accuracy_xgb = float(np.sum(preds_xgb == np.array(y_test)))/len(np.array(y_test))
print('Accuracy de XGBoost: ', accuracy_xgb)
accuracy_lr = clf_lr.score(X_test, y_test)
print('Accuracy de Logistic Regression: ', accuracy_lr)

Accuracy de XGBoost:  0.5286691215149921
Accuracy de Logistic Regression:  0.5407680168332457


In [17]:
confusion_matrix(y_test, preds_xgb)

array([[300,  95, 231],
       [113, 410, 131],
       [208, 118, 295]])

In [18]:
data=xgb.DMatrix(X, label=y)

In [19]:
#Performing cross-validation
#cv_results = xgb.cv(data, num_boost_round=10, nfold=3,metrics='logloss', seed=42)

#Printing the results
#print(cv_results)